In [11]:
import os
import sys; sys.path.append("~/HHbbgg_ETH_devel/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)
import preprocessing_utils as preprocessing
reload(preprocessing)
import plotting_utils as plotting
reload(plotting)
import optimization_utils as optimization
reload(optimization)
import postprocessing_utils as postprocessing
reload(postprocessing)


<module 'postprocessing_utils' from '/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/Training/python/postprocessing_utils.py'>

In [12]:
reload(utils)
reload(preprocessing)
reload(plotting)
reload(optimization)
reload(postprocessing)
ntuples = '20170620'
# "%" sign allows to interpret the rest as a system command
%env data=$utils.IO.ldata$ntuples
files = ! ls $data | sort -t_ -k 3 -n
#pick only SM
signal = [s for s in files if "GluGluToHHTo2B2G_node_SM" in s]
diphotonJets = [s for s in files if "DiPhotonJetsBox_" in s]
gJets_lowPt = [s for s in files if "GJet_Pt-20to40" in s]
gJets_highPt = [s for s in files if "GJet_Pt-40" in s]
ggH = [s for s in files if "GluGluHToGG" in s]
vbf = [s for s in files if "VBFHToGG" in s]
VH = [s for s in files if "VHToGG" in s]
bbH = [s for s in files if "bbHToGG_M-125_4FS_yb2" in s] #is this correct?
ttH = [s for s in files if "ttHToGG" in s]

Data= [s for s in files if "DoubleEG" in s]

utils.IO.add_signal(ntuples,signal,1)
print diphotonJets
utils.IO.add_background(ntuples,diphotonJets,-1)
utils.IO.add_background(ntuples,gJets_lowPt,-2)
utils.IO.add_background(ntuples,gJets_highPt,-2)
utils.IO.add_background(ntuples,ggH,-3)
utils.IO.add_background(ntuples,vbf,-4)
utils.IO.add_background(ntuples,VH,-5)
utils.IO.add_background(ntuples,bbH,-6)
utils.IO.add_background(ntuples,ttH,-7)

nBkg = len(utils.IO.backgroundName)

utils.IO.add_data(ntuples,Data,-10)

#add all nodes
nodes = []
for i in range(2,14):
    nodes.append([s for s in files if "GluGluToHHTo2B2G_node_"+str(i) in s])
for i in range(len(utils.IO.backgroundName),len(utils.IO.backgroundName)+len(nodes)):
    utils.IO.add_background(ntuples,nodes[i-nBkg],-i)
    
    
    

for i in range(len(utils.IO.backgroundName)):        
    print "using background file n."+str(i)+": "+utils.IO.backgroundName[i]
for i in range(len(utils.IO.signalName)):    
    print "using signal file n."+str(i)+": "+utils.IO.signalName[i]
print "using data file: "+ utils.IO.dataName[0]


env: data=/mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620
['output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root']
using background file n.0: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa.root
using background file n.1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-20to40_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.2: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GJet_Pt-40toInf_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8.root
using background file n.3: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_GluGluHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.4: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/20170620/output_VBFHToGG_M-125_13TeV_powheg_pythia8.root
using background file n.5: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH/root_files/201706

In [13]:
#use noexpand for root expressions, it needs this file https://github.com/ibab/root_pandas/blob/master/root_pandas/readwrite.py
#all the branches you want to save in new trees
#mva variables
branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg)'.split(",")
branch_names += 'noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()'.split(",")
branch_names += 'customLeadingPhotonIDMVA,customSubLeadingPhotonIDMVA,leadingPhotonSigOverE,subleadingPhotonSigOverE,sigmaMOverMDecorr,PhoJetMinDr'.split(",")
#mva output
#branch_names = 'HHTagger2017'.split(",")
#additional variables
#branch_names += 'noexpand:diphotonCandidate.M(),noexpand:dijetCandidate.M(),MX,isSignal'.split(",")


#branch_names = 'leadingJet_bDis,subleadingJet_bDis,noexpand:fabs(CosThetaStar_CS),noexpand:fabs(CosTheta_bb),noexpand:fabs(CosTheta_gg),noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M(),noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()'.split(",")
branch_names = [c.strip() for c in branch_names]
print branch_names

import pandas as pd
import root_pandas as rpd

# no need to shuffle here, we just count events
preprocessing.set_signals_and_backgrounds("bbggSelectionTree",branch_names,shuffle=False)
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=preprocessing.set_variables(branch_names)

X_data,y_data,weights_data = preprocessing.set_data("bbggSelectionTree",branch_names)
X_data,y_data,weights_data = preprocessing.clean_signal_events_single_dataset(X_data,y_data,weights_data)

#bbggTrees have by default signal and CR events, let's be sure that we clean it
X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig=preprocessing.clean_signal_events(X_bkg,y_bkg,weights_bkg,X_sig,y_sig,weights_sig)

#np.savetxt('/tmp/micheli/xbkg.txt',X_bkg)
#preprocessing.adjust_and_compress(utils.IO.background_df[0]).to_csv('/tmp/micheli/bkg.csv')


['leadingJet_bDis', 'subleadingJet_bDis', 'noexpand:fabs(CosThetaStar_CS)', 'noexpand:fabs(CosTheta_bb)', 'noexpand:fabs(CosTheta_gg)', 'noexpand:diphotonCandidate.Pt()/diHiggsCandidate.M()', 'noexpand:dijetCandidate.Pt()/diHiggsCandidate.M()', 'customLeadingPhotonIDMVA', 'customSubLeadingPhotonIDMVA', 'leadingPhotonSigOverE', 'subleadingPhotonSigOverE', 'sigmaMOverMDecorr', 'PhoJetMinDr']


In [41]:
# load the model from disk
from sklearn.externals import joblib
loaded_model = joblib.load(os.path.expanduser('~/HHbbgg_ETH_devel/Training/output_files/allMC_resWeighting_F_noDR_minDRGJet.pkl'))

bkg = []
for i in range(0,len(utils.IO.backgroundName)-1): 
    print "bkg n:"+str(i)
    print X_bkg[y_bkg ==-i-1]
    bkg.append(X_bkg[y_bkg ==-i-1])




Y_pred_sig = loaded_model.predict_proba(X_sig)[:,loaded_model.n_classes_-1].astype(np.float64)
Y_pred_bkg = []
for i in range(0,len(utils.IO.backgroundName)-1):  
    print i
    Y_pred_bkg.append(loaded_model.predict_proba(bkg[i])[:,loaded_model.n_classes_-1].astype(np.float64))

Y_pred_data = loaded_model.predict_proba(X_data)[:,loaded_model.n_classes_-1].astype(np.float64)
print Y_pred_data 

bkg n:0
[[ 0.45153  0.52787  0.98295 ...,  0.00787  0.006    2.48915]
 [ 0.06329  0.61661  0.07118 ...,  0.03301  0.01714  0.80693]
 [ 0.24183  0.25859  0.8145  ...,  0.01154  0.00945  0.53341]
 ..., 
 [ 0.2427   0.17762  0.78432 ...,  0.01374  0.00874  2.24835]
 [ 0.20376  0.52999  0.76422 ...,  0.00597  0.0105   1.77568]
 [ 0.27768  0.54684  0.57091 ...,  0.0065   0.0105   0.56502]]
bkg n:1
[[ 0.8273   0.29957  0.92729 ...,  0.01707  0.0105   1.8469 ]
 [ 0.1462   0.1667   0.9743  ...,  0.00673  0.01544  1.36054]
 [ 0.22721  0.38836  0.71193 ...,  0.04957  0.02584  1.04707]
 ..., 
 [ 0.52874  0.53695  0.97045 ...,  0.03536  0.01718  2.73914]
 [ 0.16499  0.14157  0.98683 ...,  0.01581  0.0105   2.28367]
 [ 0.55322  0.55248  0.98985 ...,  0.03709  0.01846  3.04915]]
bkg n:2
[[ 0.09629  0.20702  0.575   ...,  0.00703  0.006    2.02656]
 [ 0.44396  0.67673  0.98075 ...,  0.0166   0.0105   1.82617]
 [ 0.76905  0.96456  0.79286 ...,  0.00997  0.006    1.0459 ]
 ..., 
 [ 0.26033  0.41316  0.

In [15]:
#define MVA cut and additional variables needed
import os
additionalCut_names = 'noexpand:diphotonCandidate.M(),noexpand:dijetCandidate.M(),MX,isSignal'.split(",")
#additionalCut_names += 'HHTagger2017'.split(",")
outTag = '20180108_test'
outDir=os.path.expanduser("~/HHbbgg_ETH_devel/outfiles/"+outTag)
if not os.path.exists(outDir):
    os.mkdir(outDir)

In [16]:
sig_count_df = rpd.read_root(utils.IO.signalName[0],"bbggSelectionTree", columns = branch_names+additionalCut_names)
preprocessing.define_process_weight(sig_count_df,utils.IO.sigProc[0],utils.IO.signalName[0])

#nTot is a multidim vector with all additional variables, dictVar is a dictionary associating a name of the variable
#to a position in the vector
nTot,dictVar = postprocessing.stackFeatures(sig_count_df,branch_names+additionalCut_names)
#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "Y_pred"
print Y_pred_sig.shape

#it's a stupid way to save also mva output after cuts, more efficient solutions should be found
Y_pred_total = np.column_stack((nCleaned,Y_pred_sig))

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_sig)

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.signalName[0].split("/")[len(utils.IO.signalName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_sig,nameTree="reducedTree_sig")


Y_pred
(18646,)


In [17]:
## do gJets not in the loop since they have two samples for one process
bkg_1_count_df = rpd.read_root(utils.IO.backgroundName[1],"bbggSelectionTree", columns = branch_names+additionalCut_names)
preprocessing.define_process_weight(bkg_1_count_df,utils.IO.bkgProc[1],utils.IO.backgroundName[1])

crazySF_20=25
nTot,dictVar = postprocessing.stackFeatures(bkg_1_count_df,branch_names+additionalCut_names,SF=crazySF_20)

print nTot.shape

bkg_2_count_df = rpd.read_root(utils.IO.backgroundName[2],"bbggSelectionTree", columns = branch_names+additionalCut_names)
preprocessing.define_process_weight(bkg_2_count_df,utils.IO.bkgProc[2],utils.IO.backgroundName[2])

crazySF_40=3
nTot_2,dictVar = postprocessing.stackFeatures(bkg_2_count_df,branch_names+additionalCut_names,SF=crazySF_40)


nTot_3 = np.concatenate((nTot,nTot_2))

print nTot_3.shape
#apply isSignal cleaning
nCleaned = nTot_3[np.where(nTot_3[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape

processPath=(os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root").replace("_Pt-20to40","")
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[1])

processPath=(os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[1].split("/")[len(utils.IO.backgroundName[1].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root").replace("_Pt-20to40","")
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[1],nameTree="reducedTree_bkg_2")




(2590, 18)
(63259, 18)
nCleaned
(22278, 18)


In [18]:
for iProcess in range(0,len(utils.IO.backgroundName)):
    ##gJets which are two samples for one process are skipped
    iSample=iProcess
    if iProcess == 1 or iProcess ==2:
            continue
    if iProcess > 2:
        iSample = iProcess-1
    
    print "Processing sample: "+str(iProcess)
    bkg_count_df = rpd.read_root(utils.IO.backgroundName[iProcess],"bbggSelectionTree", columns = branch_names+additionalCut_names)
    preprocessing.define_process_weight(bkg_count_df,utils.IO.bkgProc[iProcess],utils.IO.backgroundName[iProcess])

    crazySF=1
    ##scale diphoton + jets
    if iProcess == 0:
        crazySF=1.45
    nTot,dictVar = postprocessing.stackFeatures(bkg_count_df,branch_names+additionalCut_names,SF=crazySF)


    nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
    print "nCleaned"
    print nCleaned.shape

    processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[iProcess].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
    postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[iSample])
    
    processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.backgroundName[iProcess].split("/")[len(utils.IO.backgroundName[7].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root"
    if "GluGluToHHTo2B2G_node_"in processPath:
        treeName = "reducedTree_sig_node_"+str(iProcess-6)
    else:
        treeName = "reducedTree_bkg_"+str(iProcess)
    
    postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_bkg[iSample],nameTree=treeName)

Processing sample: 0
nCleaned
(1900684, 18)
Processing sample: 3
nCleaned
(56382, 18)
Processing sample: 4
nCleaned
(76431, 18)
Processing sample: 5
nCleaned
(57455, 18)
Processing sample: 6
nCleaned
(106395, 18)
Processing sample: 7
nCleaned
(268863, 18)
Processing sample: 8
nCleaned
(18365, 18)
Processing sample: 9
nCleaned
(19535, 18)
Processing sample: 10
nCleaned
(18466, 18)
Processing sample: 11
nCleaned
(16718, 18)
Processing sample: 12
nCleaned
(18934, 18)
Processing sample: 13
nCleaned
(19087, 18)
Processing sample: 14
nCleaned
(15902, 18)
Processing sample: 15
nCleaned
(16949, 18)
Processing sample: 16
nCleaned
(17490, 18)
Processing sample: 17
nCleaned
(18954, 18)
Processing sample: 18
nCleaned
(20479, 18)
Processing sample: 19
nCleaned
(21139, 18)


In [19]:
data_count_df = rpd.read_root(utils.IO.dataName[0],"bbggSelectionTree", columns = branch_names+additionalCut_names)

nTot,dictVar = postprocessing.stackFeatures(data_count_df,branch_names+additionalCut_names,isData=1)


#apply isSignal cleaning
nCleaned = nTot[np.where(nTot[:,dictVar['weight']]!=0),:][0]
print "nCleaned"
print nCleaned.shape
print Y_pred_data.shape

#save preselection data
processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data)

processPath=os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+utils.IO.dataName[0].split("/")[len(utils.IO.dataName[0].split("/"))-1].replace("output_","").replace(".root","")+"_preselection_diffNaming"+".root"
postprocessing.saveTree(processPath,dictVar,nCleaned,Y_pred_data,nameTree="reducedTree_bkg")

nCleaned
(92390, 18)
(92390,)


In [40]:
#from subprocess import call
#call(["hadd","daje.root *diffNaming.root"])
print "cd "+os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag
os.system('hadd '+ os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+'Total_preselection_diffNaming.root '+ os.path.expanduser('~/HHbbgg_ETH_devel/outfiles/')+outTag+'/'+'*diffNaming.root')

cd /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/outfiles/20180108_test


0

hadd Target file: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/outfiles/20180108_test/Total_preselection_diffNaming.root
hadd compression setting for all ouput: 1
hadd Source file 1: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/outfiles/20180108_test/bbHToGG_M-125_4FS_yb2_13TeV_amcatnlo_preselection_diffNaming.root
hadd Source file 2: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/outfiles/20180108_test/DiPhotonJetsBox_MGG-80toInf_13TeV-Sherpa_preselection_diffNaming.root
hadd Source file 3: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/outfiles/20180108_test/DoubleEG_preselection_diffNaming.root
hadd Source file 4: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/outfiles/20180108_test/GJet_DoubleEMEnriched_MGG-80toInf_TuneCUETP8M1_13TeV_Pythia8_preselection_diffNaming.root
hadd Source file 5: /mnt/t3nfs01/data01/shome/micheli/HHbbgg_ETH_devel/outfiles/20180108_test/GluGluHToGG_M-125_13TeV_powheg_pythia8_preselection_diffNaming.root
hadd Source file 6: /mnt/t3nf